In [1]:
import pandas

titles = pandas.read_csv("titles.tsv", sep='\t')
ratings = pandas.read_csv("ratings.tsv", sep='\t')



C:\Anaconda\envs\phoenix\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [33]:
titles.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
11059,tt0011216,movie,Spanish Fiesta,La fête espagnole,0,2019,\N,67,Drama
11636,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,0,2019,\N,\N,"Action,Crime"
16657,tt0016906,movie,Frivolinas,Frivolinas,0,2014,\N,80,"Comedy,Musical"
61124,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,\N,70,Drama
64098,tt0065392,movie,Bucharest Memories,Amintiri bucurestene,0,2020,\N,\N,Documentary


In [37]:
titles = titles[titles.startYear != '\\N']
titles = titles[titles.titleType == "movie"]
titles["startYear"] = titles["startYear"].astype(int)
titles = titles[titles.startYear > 2009]

movies = open("movies_NEW.dat")
movie_checker = []
for line in movies:
    movie_checker.append(line.split(";")[1].strip())


combined = pandas.merge(titles, ratings, on="tconst")

movie_df = pandas.DataFrame(columns=["tconst","title","genre","average_rating","numvotes","startYear"] )

data = [titles["tconst"], titles["primaryTitle"]]
headers = ["tconst", "title"]
ID = pandas.concat(data, axis = 1, keys=headers)

tconst = []
movie_name = []


for i in range(len(movie_checker)):
    if movie_checker[i] in ID["title"].tolist():
        tconst.append(ID[ID["title"]==movie_checker[i]]["tconst"].values[0])
        movie_name.append(movie_checker[i])


for i in range(len(tconst)):
    if tconst[i] in combined["tconst"].tolist():
        movie_df = movie_df.append({"tconst":tconst[i],
                                    "title":combined[combined["tconst"]==tconst[i]]["primaryTitle"].values[0],
                                    "genre":combined[combined["tconst"]==tconst[i]]["genres"].values[0],
                                    "average_rating":combined[combined["tconst"]==tconst[i]]["averageRating"].values[0],
                                    "numvotes":combined[combined["tconst"]==tconst[i]]["numVotes"].values[0],
                                    "startYear":combined[combined["tconst"]==tconst[i]]["startYear"].values[0]
                                    }, ignore_index=True)


movie_df

,tconst,title,genre,average_rating,numvotes,startYear
0,tt1201607,Harry Potter and the Deathly Hallows: Part 2,"Adventure,Fantasy,Mystery",8.1,808813,2011
1,tt1399103,Transformers: Dark of the Moon,"Action,Adventure,Sci-Fi",6.2,395497,2011
2,tt1324999,The Twilight Saga: Breaking Dawn - Part 1,"Adventure,Drama,Fantasy",4.9,232357,2011
3,tt1411697,The Hangover Part II,Comedy,6.5,479995,2011
4,tt1298650,Pirates of the Caribbean: On Stranger Tides,"Action,Adventure,Fantasy",6.6,500148,2011
...,...,...,...,...,...,...
2014,tt10832274,Swan Song,Drama,7.0,878,2021
2015,tt9352356,The Djinn,"Horror,Thriller",4.7,3160,2021
2016,tt7765850,Walking with Herb,Comedy,5.2,117,2021
2017,tt9738716,The World to Come,"Drama,Romance",6.2,6030,2020


In [39]:
movie_df.to_csv("IMDB.csv")